## 1.读取annotation，然后整理成用于TSMatch训练的train.pkl


In [39]:
import pandas as pd
import numpy as np
import scipy
import pickle

def _calc_bev_pos(src_point_M, warpMatrix):
    one_column = np.ones((len(src_point_M), 1))
    src_point_M = np.column_stack((src_point_M, one_column))
    tar_pt = np.matmul(src_point_M, warpMatrix.T)
    tar_pt[:, 0] = tar_pt[:, 0] / tar_pt[:, 2]
    tar_pt[:, 1] = tar_pt[:, 1] / tar_pt[:, 2]
    return tar_pt[:, 0:2]
    
def calc_bev_pos(cam_gt_p, warpMatrix):
    cam_gt = pd.read_csv(cam_gt_p, delimiter=',')
    cam_gt.columns = ['frame', 'ID', 'left', 'top', 'width', 'height', 'conf', '-1', '-1', '-1']
    cam_gt = cam_gt.drop('-1', axis=1)
    # 读取width和height
    cam_gt_sub = cam_gt[['left', 'top', 'width', 'height']].values
    cam_gt_sub_x = cam_gt_sub[:, 0] + cam_gt_sub[:, 2] / 2
    cam_gt_sub_y = cam_gt_sub[:, 1] + cam_gt_sub[:, 3] / 4 * 3
    print(cam_gt_sub_y.shape)
    cam_gt['px'] = cam_gt_sub_x
    cam_gt['py'] = cam_gt_sub_y
    tar_pt_M = _calc_bev_pos(cam_gt[['px', 'py']].values, warpMatrix)
    cam_gt[['bx', 'by']] = tar_pt_M
    # print(cam_gt)
    return cam_gt

def bspline_interpolate_t(bev, min_thr=5, point_nums=30):
    if max(bev.shape) < min_thr: 
        return bev
    min_w, max_w = min(bev[1]), max(bev[1])
    min_h, max_h = min(bev[2]), max(bev[2])
    width = max_w - min_w
    height = max_h - min_h
    if width > height:
        x_tmp = bev[1]
        y_tmp = bev[2]
    else:
        x_tmp = bev[2]
        y_tmp = bev[1]
    # sort xtmp ytmp
    x_tmp_sorted = []
    y_tmp_sorted = []

    sorted_idx = np.argsort(x_tmp)
    ts, te = min(bev[0]), max(bev[0])
    if bev[0, sorted_idx[0]] - ts > bev[0, sorted_idx[-1]] - ts: # ts和排序后的最小值对应的time match，则不用颠倒顺序,反之要颠倒顺序
        ts, te = te, ts
    for idx in sorted_idx:
        x_tmp_sorted.append(x_tmp[idx])
        y_tmp_sorted.append(y_tmp[idx])
    # print(len(x_tmp_sorted))
    spl = scipy.interpolate.splrep(x=x_tmp_sorted, y=y_tmp_sorted, s=1000*len(x_tmp_sorted)) # 为保证平滑，s越大越好，但是建议和输入点的个数相关。default s= len(x_tmp_sorted) - math.sqrt(2*len(x_tmp_sorted))
    bev_x_bs = np.linspace(x_tmp_sorted[0], x_tmp_sorted[-1], point_nums)
    time_bs = np.linspace(ts, te, point_nums)
    bev_y_bs = scipy.interpolate.splev(bev_x_bs, spl)
    if width > height:
        return np.vstack((time_bs, bev_x_bs, bev_y_bs))
    else:
        return np.vstack((time_bs, bev_y_bs, bev_x_bs))
    

# gt
c001_gt_p = r'/aic/datasets/AIC22_Track1_MTMC_Tracking/train/S01/c001/gt/gt.txt'
c002_gt_p = r'/aic/datasets/AIC22_Track1_MTMC_Tracking/train/S01/c002/gt/gt.txt'
c003_gt_p = r'/aic/datasets/AIC22_Track1_MTMC_Tracking/train/S01/c003/gt/gt.txt'
c004_gt_p = r'/aic/datasets/AIC22_Track1_MTMC_Tracking/train/S01/c004/gt/gt.txt'

# 相机的投影矩阵
c001_hom_p = r'trans_cfg/c001_trans_bev.npy'
c002_hom_p = r'trans_cfg/c002_trans_bev.npy'
c003_hom_p = r'trans_cfg/c003_trans_bev.npy'
c004_hom_p = r'trans_cfg/c004_trans_bev.npy'
c001_hom = np.load(c001_hom_p) 
c002_hom = np.load(c002_hom_p)
c003_hom = np.load(c003_hom_p)
c004_hom = np.load(c004_hom_p)

# S01_gt merge from c001,2,3,4
# 还应该做一下帧同步，因为四个摄像头在初始帧不是同一时间，
# c001_gt_pd = c001_gt_pd['frame'] + 1
fps=10
timestamp = [0, 1.640, 2.049, 2.177, 2.235]
timestamp = list(map(lambda x: x*10, timestamp))


c001_gt_pd = calc_bev_pos(c001_gt_p, c001_hom) # add bev position column
c001_gt_pd['frame'] = (c001_gt_pd['frame'] + timestamp[0]) / 10
c001_gt_pd.insert(loc=0, column='cam', value='1') # merge

c002_gt_pd = calc_bev_pos(c002_gt_p, c002_hom)
c002_gt_pd['frame'] = (c002_gt_pd['frame'] + timestamp[1]) / 10
c002_gt_pd.insert(loc=0, column='cam', value='2')

c003_gt_pd = calc_bev_pos(c003_gt_p, c003_hom)
c003_gt_pd['frame'] = (c003_gt_pd['frame'] + timestamp[2]) / 10
c003_gt_pd.insert(loc=0, column='cam', value='3')

c004_gt_pd = calc_bev_pos(c004_gt_p, c004_hom)
c004_gt_pd['frame'] = (c004_gt_pd['frame'] + timestamp[3]) / 10
c004_gt_pd.insert(loc=0, column='cam', value='4')

S01_pd = pd.concat([c001_gt_pd, c002_gt_pd, c003_gt_pd, c004_gt_pd], axis = 0, ignore_index = True, join = "outer")

# 将gt的轨迹点数量统一shape

# 用字典存，
'''
{
    tid: array([cam, frame, ])
}
'''
output = {}
S01_ids = np.unique(S01_pd['ID'].values)
for idx, id in enumerate(S01_ids):
    S01_tid_pd = S01_pd[S01_pd['ID'] == id]
    cam_ids = np.unique(S01_tid_pd['cam'].values)
    for cam in cam_ids:
        S01_cid_tid_pd = S01_tid_pd[S01_tid_pd['cam']==cam]
        bev = S01_cid_tid_pd[['frame','bx', 'by']].values.T
        bev_bs = bspline_interpolate_t(bev, min_thr=5, point_nums=30) # frame也进行了插值 frame, bx, by
        time_tmp = np.ones((1, bev_bs.shape[1]))*int(cam)
        bev_bs = np.vstack((time_tmp, bev_bs)).T
        
        # tmp = np.hstack((S01_cid_tid_pd, bev_bs))
        if bev_bs.shape[0] < 30:
            continue
        if id in output:
            output[id] = np.vstack((output[id], bev_bs))
        else:
            output[id] = bev_bs


(3311,)
(6323,)
(6679,)
(6015,)


In [44]:
output[1]

array([[   2.  ,   11.84,  146.83, -304.71],
       [   2.  ,   11.5 ,  182.1 , -282.95],
       [   2.  ,   11.15,  217.09, -261.19],
       [   2.  ,   10.81,  251.66, -239.44],
       [   2.  ,   10.46,  285.66, -217.68],
       [   2.  ,   10.12,  318.95, -195.93],
       [   2.  ,    9.77,  351.39, -174.17],
       [   2.  ,    9.43,  382.83, -152.41],
       [   2.  ,    9.08,  413.13, -130.66],
       [   2.  ,    8.74,  442.15, -108.9 ],
       [   2.  ,    8.39,  469.74,  -87.14],
       [   2.  ,    8.05,  495.76,  -65.39],
       [   2.  ,    7.7 ,  520.06,  -43.63],
       [   2.  ,    7.36,  542.5 ,  -21.87],
       [   2.  ,    7.01,  562.93,   -0.12],
       [   2.  ,    6.67,  581.22,   21.64],
       [   2.  ,    6.32,  597.22,   43.4 ],
       [   2.  ,    5.98,  610.78,   65.15],
       [   2.  ,    5.63,  621.76,   86.91],
       [   2.  ,    5.29,  630.02,  108.67],
       [   2.  ,    4.94,  635.41,  130.42],
       [   2.  ,    4.6 ,  637.79,  152.18],
       [  

## 2.保存用于训练的轨迹pkl

In [46]:
with open('./data/tracklets_train.pkl', 'wb') as f:
    pickle.dump(output, f)

In [ ]:
A_pd[0] =  A_pd[0] + 1
A_pd